In [28]:
! git clone https://github.com/Turbo87/utm
%pip install /content/utm/
%pip install geopandas
%pip install geojson
import pandas as pd
import requests
import json
import numpy as np
import utm
import matplotlib.pyplot as plt
from shapely.geometry import MultiPolygon, Polygon
from shapely.ops import cascaded_union
from shapely import wkt
from scipy.interpolate import interp1d
import geojson

fatal: destination path 'utm' already exists and is not an empty directory.
Processing ./utm
  Created wheel for utm: filename=utm-0.7.0-cp37-none-any.whl size=6302 sha256=3d7c66614baa7da45f712232d8da080fe59c2cc42540e2d211f650d99b5903b7
  Stored in directory: /tmp/pip-ephem-wheel-cache-j820lme5/wheels/16/04/70/b9fc0e9cc286647998c5b2ed47eaa4a729262fa1b1ccf3dceb
Successfully built utm
  Found existing installation: utm 0.7.0
    Uninstalling utm-0.7.0:
      Successfully uninstalled utm-0.7.0


In [32]:
def get_influence_polygon(inputJson):
    lat = inputJson["coordinates"][0]
    print(lat)
    lon = inputJson["coordinates"][1]
    print(lon)

    resp = requests.get('http://api.openweathermap.org/data/2.5/weather?lat=' + str(lat) + '&lon=' + str(lon) + '&appid={}')
    if resp.status_code != 200:
        # This means something went wrong.
        raise ApiError('GET /tasks/ {}'.format(resp.status_code))

    #print(resp.text)

    ##Prettify JSON and get Wind Information
    parsed = json.loads(resp.text)
    wind = parsed["wind"]
    wind_speed = parsed["wind"]["speed"]
    wind_dir = parsed["wind"]["deg"]
    #print(wind_speed)
    #print(wind_dir)

    ##Create wind cones
    #Recalculation of movement in km
    #Wind speed is calculated from m/s to km/h via *3.6
    #Fires move 1/10th the km/h of the wind speed (see paper above), thus /10
    wind_speed_move = (float(wind_speed)*3.6)/10 
    #print(wind_speed_move)

    #Recalculation from geographic to arithmetic degrees
    #+180 because the fire goes away from the wind, not into the same direction
    dir_new = (-(wind_dir+180)+450)%360
    #print(dir_new)

    #Calculate x and y
    dx=wind_speed_move*np.cos(np.radians(dir_new))
    #print(dx)
    dy=wind_speed_move*np.sin(np.radians(dir_new))
    #print(dy)

    #Conversion to degrees latitude and longitude
    #The approximate conversions are:
    #Latitude: 1 deg = 110.574 km -> deg = x/110.574
    #Longitude: 1 deg = 111.320*cos(latitude) km -> deg = x/111.320

    #conversion to UTM ('meters')
    utm_coords = utm.from_latlon(lat, lon)

    #add x and y in meters
    newlat=utm_coords[0]+(dx*1000)
    newlon=utm_coords[1]+(dy*1000)

    #Return to WGS84
    pos2 = utm.to_latlon(newlat,newlon,utm_coords[2],utm_coords[3])
    #print(pos2)

    # x, y: coords of forecast position
    y = [lat, pos2[0],47.81765021489975,47.831134971598395]
    x = [lon, pos2[1],13.046951293945312,13.071155548095703]
    # r: radius of uncertainty
    r = [0, 0.001, 0.002, 0.003]
    hours = [0, 1, 2, 3]
    # interpolate
    points_num = 100
    interp_hours = np.linspace(min(hours), max(hours), points_num)
    x = interp1d(hours, x, kind='cubic')(interp_hours)
    y = interp1d(hours, y, kind='cubic')(interp_hours)
    r = interp1d(hours, r, kind='linear')(interp_hours)
    # make polygon
    thetas = np.linspace(0, 2 * np.pi, 360)
    polygon_x = x[:,None] + r[:,None] * np.sin(thetas)
    polygon_y = y[:,None] + r[:,None] * np.cos(thetas)

    # circles
    ps = [Polygon(i) for i in np.dstack((polygon_x, polygon_y))]

    # list of convex hulls of subsequent circles
    n = range(len(ps)-1)
    convex_hulls = [MultiPolygon([ps[i], ps[i+1]]).convex_hull for i in n]

    # Final polygon
    polygons = cascaded_union(convex_hulls)

    #Convert to WKT to convert then to geojson
    g1=wkt.loads(str(polygons))
    g2=geojson.Feature(properties={"Name": "Betroffenheitspolygon"},geometry=g1)

    #print(g2)
    return g2.geometry

In [33]:
#input_coords = '{"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"Point","coordinates":[13.033218383789062,47.79355334155344]}}]}'
input_coords=geojson.Point((47.79355334155344, 13.033218383789062))
print(input_coords)

{"coordinates": [47.793553, 13.033218], "type": "Point"}


In [34]:
polygon=get_influence_polygon(input_coords)

47.793553
13.033218


In [35]:
polygon

{"coordinates": [[[13.028436, 47.774773], [13.028425, 47.774773], [13.028414, 47.774773], [13.028403, 47.774774], [13.028392, 47.774774], [13.028381, 47.774775], [13.02837, 47.774777], [13.028359, 47.774778], [13.028348, 47.77478], [13.028337, 47.774781], [13.028326, 47.774783], [13.028315, 47.774785], [13.028304, 47.774788], [13.028293, 47.77479], [13.028282, 47.774793], [13.028271, 47.774796], [13.028261, 47.774799], [13.02825, 47.774802], [13.028239, 47.774806], [13.028229, 47.774809], [13.028218, 47.774813], [13.028208, 47.774817], [13.028198, 47.774821], [13.028187, 47.774826], [13.028177, 47.77483], [13.028167, 47.774835], [13.028157, 47.77484], [13.028147, 47.774845], [13.028137, 47.77485], [13.028128, 47.774855], [13.028118, 47.774861], [13.028109, 47.774867], [13.028099, 47.774873], [13.02809, 47.774879], [13.028081, 47.774885], [13.028071, 47.774891], [13.028062, 47.774898], [13.028054, 47.774905], [13.028045, 47.774912], [13.028036, 47.774919], [13.028028, 47.774926], [13.02

In [27]:
input_coords=geojson.Point((47.80, 13.04))

lat = input_coords["coordinates"][0]
print(lat)
lon = input_coords["coordinates"][1]
print(lon)

47.8
47.8
13.04
